In [1]:
from modules.utils import sorter, cleaner
from modules.kerasfns import create_model, model_fitter, pred_perf, hist_acc
from modules.preprocess import loader, sample_print

from keras.regularizers import l2
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
#Clean & split image data into folders 
[cleaner(i) for i in ["train","test","validation"]]

sorter(.6,.2,.2)


In [ ]:
# Preparing train / test / valid datasets:

train = loader("train",
augmentation = True, 
rotation_range= 45, 
width_shift_range=.2, 
height_shift_range=.2, 
shear_range = 0.2, 
zoom_range = 0.2, 
horizontal_flip = True)

valid = loader("validation")
test_set = loader("test", shuffle=False)

In [ ]:
sample_print(train)

In [ ]:
# MODEL TRIALS

In [ ]:
# Trial #1 - Creating the base model

model1 = create_model([32,64,128], neuron_dense=256, dropout=True)
hist1, model1 = model_fitter(model1,10,train,valid)
pred1 = pred_perf(model1,test_set)

In [ ]:
# Hyperparameter Tuning

# Trial #2 - Dropout - Disharmony!
model2 = create_model([32,64,128], neuron_dense=256 )
hist2, model2 = model_fitter(model2,10,train,valid)
pred2 = pred_perf(model2,test_set)

#As several sources and article suggest, using both pooling and dropout may actually cause disharmony between the two, and resulting with a worse model than if neither or just one of them were used. In this particular model the performance is not significantly affected by using/removing Dropout, however, as there is a slight increase in accuaracy and decrease in loss, we decided to remove Dropout.abs

# It should also be noted that Dropout layer was tested later on in the projet, this time causing a more significant decrease in model performance.

In [ ]:
#Trial #3 - Increase number of neurons in the Conv layers
model3 = create_model([64,128,256], neuron_dense=256)
hist3, model3 = model_fitter(model3,10,train,valid)
pred3 = pred_perf(model3,test_set)

#Increasing number of neurons in the conv layers caused overfitting!

In [2]:
# Trial #4 - Increase the number of conv layers (to 4)
model4 = create_model([32,64,128,128], neuron_dense=256)
hist4, model4 = model_fitter(model4,10,train,valid)
pred4 = pred_perf(model4,test_set)

#Adding hidden layers improved model performance!

In [ ]:
# Trial #5 - Increase the number of conv layers (to 5)
model5 = create_model([32,64,128,128,128], neuron_dense=256)
hist5, model5 = model_fitter(model5,10,train,valid)
pred5 = pred_perf(model5,test_set)

# Adding another hidden layer did not improve the model performance, and although the model is able to reach %84 val_acc in around 5 epochs, model then overfits and not able to improve val_acc anymore.

In [ ]:
# Trial #6 - Set number of neurons in the Dense layer (to 128)
model6 = create_model([32,64,128,128])
hist6, model6 = model_fitter(model6,10,train,valid)
pred6 = pred_perf(model6,test_set)

# After several examples and research on the subject, equalling the number of neurons in the last convolution layer and the dense layer seems to be a common concept in CNNs. Therefore, the neuron number in the Dense layer was set to 128 which actually did improve model performance.

In [ ]:
# Trial #7 - Adding kernel_initializer
model7 = create_model([32,64,128,128], kernel_initializer='he_uniform')
hist7, model7 = model_fitter(model7,10,train,valid)
pred7 = pred_perf(model7,test_set)

# Using he_uniform or he_normal as the kernel_initializers actually improved model performance slightly. Howeever, it seems changing between the two does not have a significant difference for this model. 

In [4]:
#Trial #8 - Padding
model8 = create_model([32,64,128,128], kernel_initializer='he_uniform', padding="same")
hist8, model8 = model_fitter(model8,10,train,valid)
pred8 = pred_perf(model8,test_set)

# Changing padding from "valid" to "same" improved model performance as we were able to retain original image size, which would otherwise be decreased during the convolution operation.

In [ ]:
# Trial #9 - Adding kernel_regularizer
model9 = create_model([32,64,128,128], kernel_initializer='he_uniform', kernel_regularizer=l2(0.01), padding="same")
hist9, model9 = model_fitter(model9,20,train,valid)
pred9 = pred_perf(model9,test_set)

#Adding kernel regularizer actually decreased the model performance, with validation accuracy never even reaching 80% during 20 epochs. Therefore, this parameter will not be used.

In [ ]:
#With the parameter tuning the overall performance has improved and is now able to give out >80% validation accuracy over multiple trials with 10+ epochs. Therefore, we can now use callbacks to train the model longer (larger epochs) and save the best one to be submitted.

#Preparing Callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_model.hdf5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)

In [ ]:
#Trial #10 - Trying out the best model yet with 50 epochs
model10 = create_model([32,64,128,128], kernel_initializer='he_uniform', padding="same")
hist10, model10 = model_fitter(model10,50,train,valid, callbacks=[es,mc])
pred10 = pred_perf(model10,test_set)

# The iteration stops around 20-30 epochs due to early stopping callback, however the model performance seems encouraging as we are able to reach around 88% val_acc, the highest number yet! To view the overall process we can use hist_acc fundtion to plot the loss and accuracy metrics across epochs. 

In [ ]:
hist_acc(hist10)

In [ ]:
# As a final test we can reload the model and evaluate it again to test the performance of our final model one last time.

model_final = load_model("/Saved Models/best_model.hdf5")
pred_final = pred_perf(final_model)

# The predictions have a loss of 0.3382 and an accuracy of 0.8480
# By evaluating these values, we decided to use this model as the final product of this project, and can be used for threat level detection.